# CMP304 Assessment - Pokemon Image Classification 

In [2]:
import os, sys 

import random
from IPython.display import display, Image
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.preprocessing.image import array_to_img, ImageDataGenerator, img_to_array, load_img
from tensorflow.keras import models, layers


In [3]:
BATCH_SIZE = 10
IMAGE_SIZE = 200
CHANNELS = 3
EPOCHS = 10

We import a number of libraries for use in this assessment.

OLD
Next up we load the data from the .csv about each pokemon and output the data.

In [7]:
Pokemon_Data = 'PokemonArchive/pokemon.csv'
Pokemon_Count = pd.read_csv(Pokemon_Data)
Pokemon_Count.head()

# Display all of the data in the .csv
pd.set_option('display.max_rows', None)  # Set to None to display all rows
pd.set_option('display.max_columns', None)  # Set to None to display all columns

# Print the entire DataFrame
print(Pokemon_Count)

                     Name     Type1     Type2   Evolution
0               bulbasaur     Grass    Poison     ivysaur
1                 ivysaur     Grass    Poison    venusaur
2                venusaur     Grass    Poison         NaN
3              charmander      Fire       NaN  charmeleon
4              charmeleon      Fire       NaN   charizard
5               charizard      Fire    Flying         NaN
6                squirtle     Water       NaN   wartortle
7               wartortle     Water       NaN   blastoise
8               blastoise     Water       NaN         NaN
9                caterpie       Bug       NaN     metapod
10                metapod       Bug       NaN  butterfree
11             butterfree       Bug    Flying         NaN
12                 weedle       Bug    Poison      kakuna
13                 kakuna       Bug    Poison    beedrill
14               beedrill       Bug    Poison         NaN
15                 pidgey    Normal    Flying   pidgeotto
16            

We then path to the images of the pokemon and output the number of pokemon folders as well as the name of each folder.

In [5]:
# Display all images in PokemonArchive/images
# File path to images
Pokemon_Images_Dir = 'PokemonArchive/images/'

labels = os.listdir(Pokemon_Images_Dir)
nb = len(labels)
#print('Size - ', nb)
#print(labels)

OLD
Check that each pokemon in the .csv has a matching images folder

In [9]:
# Get a list of Pokémon names from the CSV file
Pokemon_Names = Pokemon_Count['Name'].tolist()

# Initialize counter variables
No_Folder_Count = 0
Folder_Count = 0

# Iterate through to check each pokemon has images
for Pokemon_Name in labels:
    Pokemon_File_Path = os.path.join(Pokemon_Images_Dir, Pokemon_Name)

    # Check if the path is a directory
    if os.path.isdir(Pokemon_File_Path):
        # Increment folder count if folder is found
        Folder_Count += 1

        # Get a list of files in the Pokémon folder
        Files_In_Folder = os.listdir(Pokemon_File_Path)
        
        # Flag to check if any image is found for the Pokémon
        Image_Found = False

        # Check each file in the folder
        for File_Name in Files_In_Folder:
            if File_Name.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
                # Display the image inline in the notebook
                #display(Image(filename=os.path.join(pokemon_folder_path, file_name)))
                Image_Found = True
                break  # No need to continue loop if image is found
        
        # Check if no image is found for the Pokémon
        if not Image_Found:
            print(f"No image found for {PokemonName}")
    else:
        # Increment the counter if no folder is found
        No_Folder_Count += 1
        print(f"No folder found for {Pokemon_Name}")

# Output the count of Pokémon without folders
print(f"Total number of Pokémon without folders: {No_Folder_Count}")

# Output the count of folders
print(f"Total number of Pokémon folders: {Folder_Count}")

Total number of Pokémon without folders: 0
Total number of Pokémon folders: 150


I need to:
- Normalize each image to same size (possibly)
- Generate a number of images from each
- randomly cropped pictures from each picture
- Random Rotations
- Grayscaled etc

Try to test results from setting sizes of 32x32, 64x64 and 128x128
      

In [11]:
normalized_Image_Dir = "PokemonArchive/NormalizedImages"

# Desired new dimensions for the resized images
New_Width = 128
New_Height = 128

# variable for current Pokemon
Current_Pokemon = 0

# Iterate through all 150 pokemon folders
for Pokemon_Folder in labels:
    Folder_Path = os.path.join(Pokemon_Images_Dir, Pokemon_Folder)
    #print(PokemonFolder, ":")

    # Set image count to 0 for each new folder
    Image_Count = 0
    
    # if directory exists
    if os.path.isdir(Folder_Path):
         New_Image_Folder = os.path.join(normalized_Image_Dir, Pokemon_Folder)
    # if not then create it
    if not os.path.exists(New_Image_Folder):
        os.makedirs(New_Image_Folder)


    # iterate through each image in the each pokemon folder 
    for Image_Name in os.listdir(Folder_Path):
            Image_Path = os.path.join(Folder_Path, Image_Name)

        # if file is an image format
            if Image_Path.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
                # Get File Type
                File_Type = os.path.splitext(Image_Name)[1]
                # open image to edit
                with Image.open(Image_Path) as img:
                    # Resize the image
                    New_Image = img.resize((New_Width, New_Height))
                    
                    # Rename each image to say PokemonName_Number
                    New_Name = f"{Pokemon_Folder}_{Image_Count}{File_Type}"
                    
                    # Save the resized image to the normalized folder
                    New_Image.save(os.path.join(New_Image_Folder, New_Name))
                    
                    Image_Count += 1
                
            plt.figure()
            plt.imshow(New_Image)
            plt.title(f'Pokemon: {Pokemon_Folder}')
            plt.axis('off')  # Turn off axis
            #plt.show()  # Show the plot
            plt.close()
            


Break each image into pieces and save in another folder

Create a function for Image Generation
Generate:
- Grayscale images
- Random rotations
- Random transform/crops

Pseudocode
  - Input normalized image folder, folder directory for where new images will be stored, how many images will be output
  - Iterate through each pokemon folder in NormalizedImages
  - Setup Output directory for each pokemon
  - In each folder iterate through each image

In [13]:


# Generate a function for image generation
def Generate_Image(Origin_Folder_Directory, New_Folder_Directory, Output_Per_Image):
    print("Running!")
    datagen = ImageDataGenerator(rotation_range = 40, width_shift_range = 0.25, height_shift_range = 0.25, 
                                 shear_range = 0.2, # range for image distortion on an axis
                                 zoom_range = 0.3, horizontal_flip = True, vertical_flip = True, # Zoom max range plus whether images can be flipped
                                 channel_shift_range = 10, # range for randomly shifting pixel values
                                 fill_mode = 'nearest') # How new pixels are filled - sample nearest pixel value
    
    # Get a list of each Pokémon folder
    Pokemon_Folders = os.listdir(Origin_Folder_Directory)

    for Pokemon_Folder in Pokemon_Folders:
        PokemonFolderPath = os.path.join(Origin_Folder_Directory, Pokemon_Folder)
        OutputDirectory = os.path.join(New_Folder_Directory, Pokemon_Folder)
        
        if not os.path.exists(Output_Directory):
            os.makedirs(Output_Directory)


        ## loop through all images in folder
        for image in os.listdir(Pokemon_Folder_Path):
            Image_Path = Pokemon_Folder_Path + "\\" + image

            # get image name and file type
            Image_Name, Image_File_Type = image.split(".")

            img = load_img(ImagePath)
            img = img_to_array(img)
            img = img.reshape((1,) + img.shape)
        
            Image_Count = 0
            for image in datagen.flow(img, batch_size=1, save_to_dir= OutputDirectory+"\\", save_prefix='augmented_' + PokemonFolder, save_format = ImageFileType):
                Image_Count += 1 
                if Image_Count >= Output_Per_Image:
                    print("Generated - " +str(Image_Count) + " Images for " +Image_Name)
                    break
        

In [14]:
Training_Directory = "PokemonArchive/TrainingImages"
Output_Per_Image = 10

print("Running Image Generation")

#Generate_Image(normalized_Image_Dir, Training_Directory, Output_Per_Image)


Running Image Generation


Train the model

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

# Get a list of subdirectories (classes) in the dataset directory
classes = os.listdir(Training_Directory)

# Create lists to store file paths and corresponding labels
File_Paths = []
# Placeholder for your actual labels
Labels = []

# Iterate through each class directory
for Class_Name in classes:
    Class_Directory = os.path.join(Training_Directory, Class_Name)
    # Get a list of file names in the class directory
    Class_Files = [os.path.join(Class_Directory, File) for File in os.listdir(Class_Directory)]
    # Randomly sample 150 images from each class
    #sampled_files = random.sample(class_files, 150)
    # Append file paths and labels for sampled files
    File_Paths.extend(Class_Files)
    # Assign labels to corresponding files
    Labels.extend([Class_Name] * len(Class_Files))

# Split the dataset into training and validation sets
Train_Files, Val_Files, Train_Labels, Val_Labels = train_test_split(File_Paths, Labels, test_size=0.2, random_state=42)

# Convert file paths and labels into a DataFrame
Train_Df = pd.DataFrame({'file_paths': Train_Files, 'labels': Train_Labels})
Val_Df = pd.DataFrame({'file_paths': Val_Files, 'labels': Val_Labels})

print(Train_Df['file_paths'])
print(Val_Df['file_paths'])

# Define the number of samples in training and validation sets
Train_Samples = len(Train_Files)
Validation_Samples = len(Val_Files)


# Setup CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(New_Width, New_Height, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(nb, activation='softmax'))

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Data Augmentation and Loading
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.25,
    height_shift_range=0.25,
    shear_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,
    channel_shift_range=10,
    fill_mode='nearest'
)

# Train generator
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='file_paths',
    y_col='labels',
    target_size=(New_Width, New_Height),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Validation generator
validation_generator = validation_datagen.flow_from_dataframe(
    val_df,
    x_col='file_paths',
    y_col='labels',
    target_size=(New_Width, New_Height),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_samples // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_samples // BATCH_SIZE
)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [72]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_samples // BATCH_SIZE,
    epochs=EPOCHS,  # specify additional epochs
    validation_data=validation_generator,
    validation_steps=validation_samples // BATCH_SIZE
)

Epoch 1/10
   1/1800 [..............................] - ETA: 2:29 - loss: 1.3694 - accuracy: 0.6000

C:\Users\Eon\AppData\Local\Temp\ipykernel_17012\3178778611.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


1800/1800 [==============================] - 120s 66ms/step - loss: 2.2696 - accuracy: 0.3829 - val_loss: 2.0728 - val_accuracy: 0.4311
Epoch 2/10
1800/1800 [==============================] - 119s 66ms/step - loss: 2.2042 - accuracy: 0.4042 - val_loss: 2.0933 - val_accuracy: 0.4296
Epoch 3/10
1800/1800 [==============================] - 120s 67ms/step - loss: 2.1531 - accuracy: 0.4189 - val_loss: 2.0022 - val_accuracy: 0.4520
Epoch 4/10
1800/1800 [==============================] - 119s 66ms/step - loss: 2.1282 - accuracy: 0.4182 - val_loss: 2.0537 - val_accuracy: 0.4342
Epoch 5/10
1800/1800 [==============================] - 119s 66ms/step - loss: 2.0805 - accuracy: 0.4342 - val_loss: 1.9047 - val_accuracy: 0.4718
Epoch 6/10
1800/1800 [==============================] - 118s 66ms/step - loss: 2.0562 - accuracy: 0.4399 - val_loss: 1.9933 - val_accuracy: 0.4524
Epoch 7/10
1800/1800 [==============================] - 122s 68ms/step - loss: 2.0246 - accuracy: 0.4432 - val_loss: 1.9742 - val

In [74]:
ModelDirectory = "Models/"

model.save(ModelDirectory + ".h5")

Test the model based on a full image of a pokemon - I chose Snorlax:

In [24]:
from tensorflow.keras.models import load_model

#print(classes)

model = load_model('Models.h5')
image_path = 'PokemonArchive/FinalPredictionImages/Snorlax.jpg'

# Load and preprocess your full image
def preprocess_image(image_path, target_size=(New_Width, New_Height)):
    img = Image.open(image_path)
    img = img.resize(target_size)
    img_array = np.array(img) / 255.0  # Normalize pixel values
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

# Make predictions on the full image
def predict_pokemon(image_path, model):
    preprocessed_image = preprocess_image(image_path)
    predictions = model.predict(preprocessed_image)
    # Assuming you have a list of class names
    class_names = classes
    predicted_class_index = np.argmax(predictions)
    predicted_class = class_names[predicted_class_index]
    confidence = predictions[0][predicted_class_index]
    return predicted_class, confidence

# Example usage
image_path = 'PokemonArchive/FinalPredictionImages/TestPokemon.jpg'
predicted_class, confidence = predict_pokemon(image_path, model)
print("Predicted class:", predicted_class)
print("Confidence:", confidence)


1/1 [==============================] - 0s 104ms/step
Predicted class: starmie
Confidence: 0.9998235
